# Decision Tree

In [23]:
import pandas as pd
import scipy.io
import numpy as np

In [29]:
 
def import_data(split = 0.8, shuffle=False):
    """Read in the data, split it by split percentage into train and test data, 
    and return X_train, y_train, X_test, y_test as numpy arrays"""
    col_names = ['var', 'skew', 'curt', 'entropy', 'is_fraud']
    
    data = pd.read_csv('data_banknote.csv', names=col_names)

    if shuffle:
        np.random.shuffle(data)

    X_data = np.asarray(data[col_names[:-1]])
    y_data = np.asarray(data['is_fraud'])

    X_train = X_data[0:int(X_data.shape[0] * split)]
    y_train = y_data[0:int(X_data.shape[0] * split)]
    X_test = X_data[int(X_data.shape[0] * split):]
    y_test = y_data[int(X_data.shape[0] * split):]


    print("data imported")

    return X_train, y_train, X_test, y_test

class Node:
    """Each node of decision tree will hold values such as left and right children, 
    the data and labels being split on, the threshold value & index in the dataframe for a particular feature,
    and the uncertainty measure for this node"""
    def __init__(self, data, labels, depth):
        """
        data: X data
        labels: y data
        depth: depth of tree
        """
        self.left = None
        self.right = None

        self.data = data
        self.labels = labels
        self.depth = depth

        self.threshold = None
        self.threshold_index = None
        self.feature = None
        self.label = None
        self.uncertainty = None
        
class DecisionTree:
    def __init__(self, K=5, verbose=False):
        """
        K: number of features to split on 
        """
        self.root = None
        self.K = K
        self.verbose = verbose

    def buildTree(self, data, labels):
        """Builds tree for training on data. Recursively called _buildTree"""
        self.root = Node(data, labels, 0)
        if self.verbose:
            print("Root node shape: ", data.shape, labels.shape)
        self._buildTree(self.root)

    def _buildTree(self, node):
             
        # get uncertainty measure and feature threshold
        node.uncertainty = self.get_uncertainty(node.labels)
        self.get_feature_threshold(node)
            
        index = node.data[:, node.feature].argsort()  # sort feature for return
        node.data = node.data[index]
        node.labels = node.labels[index]
        
        # check label distribution.
        label_distribution = np.bincount(node.labels)
        majority_label = node.labels[0] if len(label_distribution) == 1 else np.argmax(label_distribution)
            
        if self.verbose:
            print("Node uncertainty: %f" % node.uncertainty)
    
        # Split left and right if threshold is not the min or max of the feature or every point has the
        # same label.
        if node.threshold_index == 0 or node.threshold_index == node.data.shape[0] or \
            len(label_distribution) == 1:
            node.label = majority_label
        else:
            node.left = Node(node.data[:node.threshold_index], node.labels[:node.threshold_index], node.depth + 1)
            node.right = Node(node.data[node.threshold_index:], node.labels[node.threshold_index:], node.depth + 1)            
            node.data = None
            node.labels = None
                        
            # If in last layer of tree, assign predictions
            if node.depth == self.K:
                if len(node.left.labels) == 0:
                    node.right.label = np.argmax(np.bincount(node.right.labels))
                    node.left.label = 1 - node.right.label
                elif len(node.right.labels) == 0:
                    node.left.label = np.argmax(np.bincount(node.left.labels))
                    node.right.label = 1 - node.left.label
                else:
                    node.left.label = np.argmax(np.bincount(node.left.labels))
                    node.right.label = np.argmax(np.bincount(node.right.labels))
                return

            else: # Otherwise continue training the tree by calling _buildTree
                self._buildTree(node.left)
                self._buildTree(node.right)

    def predict(self, data_pt):
        return self._predict(data_pt, self.root)

    def _predict(self, data_pt, node):
        feature = node.feature
        threshold = node.threshold

        if node.label is not None:
            return node.label
        elif data_pt[node.feature] < node.threshold:
            return self._predict(data_pt, node.left)
        elif data_pt[node.feature] >= node.threshold:
            return self._predict(data_pt, node.right)

    def get_feature_threshold(self, node):
        """ Find the feature that gives the largest information gain. Update node.threshold, 
        node.threshold_index, and node.feature (a number representing the feature. e.g. 2nd column feature would be 1)
        """
 
        node.threshold = 0
        node.threshold_index = 0
        node.feature = 0
        max_val = 0
        
        for i in range(node.data.shape[1]): # for each feature 

            index = node.data[:, i].argsort() # sort feature for return
            node.data = node.data[index]
            node.labels = node.labels[index]

            _, index = np.unique(node.data[:, i], return_index=True)

            for j in index: # for each unique pixel

                value = self.getInfoGain(node, j)
                if value > max_val: # and node.data[j, i] != node.data[0, i] and node.data[j, i] != node.data[-1, i]:
                    max_val = value
                    node.threshold = node.data[j, i]
                    node.threshold_index = j
                    node.feature = i
            

    def getInfoGain(self, node, split_index):
        """
        Get information gain using the variables in the parameters, node.data, 
        node.labels, and node.uncertainty
        split_index: index in the feature column that you are splitting the classes on
        """
 
        return node.uncertainty - ((split_index / node.data.shape[0]) * self.get_uncertainty(node.labels[:split_index])
                                   + ((node.data.shape[0] - split_index) / node.data.shape[0]) * self.get_uncertainty(node.labels[split_index:]))

    def get_uncertainty(self, labels, metric="entropy"):
        """Get uncertainty. Implement entropy, classification, and gini error. 
        np.bincount(labels) and labels.shape might be useful here"""
        
        if labels.shape[0] == 0:
            return 1
 
        if metric == "classification":
            return 1 - np.max(np.bincount(labels) / labels.shape[0]) # classification error
        elif metric == "gini":
            return 1 - np.sum((np.bincount(labels) / labels.shape[0]) ** 2) # gini index
        else:
            count = np.bincount(labels)
            if len(count) == 1:
                return 0
            else:                
                if count[0] == 0 or count[0] == 0:
                    return 0
                else:
                    return (- (count / labels.shape[0]) * np.log2(count / labels.shape[0]) ).sum() # entropy


    def printTree(self):
        """Prints the tree including threshold value and feature name"""
        self._printTree(self.root)

    def _printTree(self, node):
        if node is not None:
            if node.label is None:
                print("\t" * node.depth, "(%d, %d)" % (node.threshold, node.feature))
            else:
                print("\t" * node.depth, node.label)
            self._printTree(node.left)
            self._printTree(node.right)

    def homework_evaluate(self, X_train, labels, X_test, y_test):
        n = X_train.shape[0]

        count = 0
        for i in range(n):
            if self.predict(X_train[i]) == labels[i]:
                count += 1

        print("The decision tree is %d percent accurate on %d training data" % ((count / n) * 100, n))

        n = X_test.shape[0]

        count = 0
        for i in range(n):
            if self.predict(X_test[i]) == y_test[i]:
                count += 1

        print("The decision tree is %d percent accurate on %d test data" % ((count / n) * 100, n))

        return count / n

In [30]:
X_train, y_train, X_test, y_test = import_data(split=0.8)

for k_value in range(10):
    print('For K = ', k_value)
    tree = DecisionTree(K=k_value, verbose=False)
    tree.buildTree(X_train, y_train)
    tree.homework_evaluate(X_train, y_train, X_test, y_test)

data imported
For K =  0
The decision tree is 84 percent accurate on 1097 training data
The decision tree is 88 percent accurate on 275 test data
For K =  1
The decision tree is 91 percent accurate on 1097 training data
The decision tree is 82 percent accurate on 275 test data
For K =  2
The decision tree is 95 percent accurate on 1097 training data
The decision tree is 96 percent accurate on 275 test data
For K =  3
The decision tree is 96 percent accurate on 1097 training data
The decision tree is 95 percent accurate on 275 test data
For K =  4
The decision tree is 99 percent accurate on 1097 training data
The decision tree is 96 percent accurate on 275 test data
For K =  5
The decision tree is 99 percent accurate on 1097 training data
The decision tree is 96 percent accurate on 275 test data
For K =  6
The decision tree is 100 percent accurate on 1097 training data
The decision tree is 96 percent accurate on 275 test data
For K =  7
The decision tree is 100 percent accurate on 1097 